Arquivo de classificação para o tráfego DoS

Matheus Figueiredo de Castro

Universidade Federal do Amazonas - Fevereiro de 2024

In [1]:
import numpy as np
import pandas as pd
import csvs

In [2]:
to_read = pd.read_csv("train70_reduced.csv", nrows=10000)   # 100000 foram selecionadas devido à limitação da máquina

print(to_read.shape)

(10000, 34)


Processamento dos dados

In [21]:
data = pd.DataFrame(to_read)

In [4]:
# divisão em duas classes

for index, row in data.iterrows():
    if row['target'] == 'dos':
        data.at[index, 'target'] = '1'
    else:
        data.at[index, 'target'] = '0'

In [5]:
print(data['target'].unique())

['0' '1']


In [10]:
# para a feature TCP Flags

for number, row in data.iterrows():                     # number represents de index of the column
    if row['tcp.flags'] == "0x00000010":                # row represents the data of the row
        data.at[number, 'tcp.flags'] = "ACK"            # the method at. update the row to the new value
    elif row['tcp.flags'] == "0x00000002":
        data.at[number, 'tcp.flags'] = "SYN"
    elif row['tcp.flags'] == "0x00000012":
        data.at[number, 'tcp.flags'] = "SYN ACK"
    elif row['tcp.flags'] == "0x00000011":
        data.at[number, 'tcp.flags'] = "FIN ACK"
    elif row['tcp.flags'] == "0x00000004":
        data.at[number, 'tcp.flags'] = "RST"
    elif row['tcp.flags'] == "0x00000014":
        data.at[number, 'tcp.flags'] = "RST ACK"
    elif row['tcp.flags'] == "0x00000018":
        data.at[number, 'tcp.flags'] = "PSH ACK"
    elif row['tcp.flags'] == "0x00000019":
        data.at[number, 'tcp.flags'] = "ZERO"

In [12]:
# para a feature MQTT Flags

for number, row in data.iterrows():                             # number represents de index of the column
    if row['mqtt.hdrflags'] == "0x00000030":                    # row represents the data of the row
        data.at[number, 'mqtt.hdrflags'] = "110000"             # the method at. update the row to the new value
    elif row['mqtt.hdrflags'] == "0x00000032":
        data.at[number, 'mqtt.hdrflags'] = "110010"
    elif row['mqtt.hdrflags'] == "0x00000040":
        data.at[number, 'mqtt.hdrflags'] = "1000000"
    elif row['mqtt.hdrflags'] == "0":
        data.at[number, 'mqtt.hdrflags'] = "0"
    elif row['mqtt.hdrflags'] == "0x00000020":
        data.at[number, 'mqtt.hdrflags'] = "100000"
    elif row['mqtt.hdrflags'] == "0x0000003a":
        data.at[number, 'mqtt.hdrflags'] = "111010"
    elif row['mqtt.hdrflags'] == "0x000000d0":
        data.at[number, 'mqtt.hdrflags'] = "11010000"
    elif row['mqtt.hdrflags'] == "0x000000c0":
        data.at[number, 'mqtt.hdrflags'] = "11000000"
    elif row['mqtt.hdrflags'] == "0x00000010":
        data.at[number, 'mqtt.hdrflags'] = "10000"
    elif row['mqtt.hdrflags'] == "0x00000090":
        data.at[number, 'mqtt.hdrflags'] = "10010000"
    elif row['mqtt.hdrflags'] == "0x00000031":
        data.at[number, 'mqtt.hdrflags'] = "110001"
    elif row['mqtt.hdrflags'] == "0x00000050":
        data.at[number, 'mqtt.hdrflags'] = "1010000"
    elif row['mqtt.hdrflags'] == "0x00000082":
        data.at[number, 'mqtt.hdrflags'] = "10000010"
    elif row['mqtt.hdrflags'] == "0x000000e0":
        data.at[number, 'mqtt.hdrflags'] = "11100000"

Reajuste de dados

In [15]:
print(list(data.keys()))

['tcp.flags', 'tcp.time_delta', 'tcp.len', 'mqtt.conack.flags', 'mqtt.conack.flags.reserved', 'mqtt.conack.flags.sp', 'mqtt.conack.val', 'mqtt.conflag.cleansess', 'mqtt.conflag.passwd', 'mqtt.conflag.qos', 'mqtt.conflag.reserved', 'mqtt.conflag.retain', 'mqtt.conflag.uname', 'mqtt.conflag.willflag', 'mqtt.conflags', 'mqtt.dupflag', 'mqtt.hdrflags', 'mqtt.kalive', 'mqtt.len', 'mqtt.msg', 'mqtt.msgid', 'mqtt.msgtype', 'mqtt.proto_len', 'mqtt.protoname', 'mqtt.qos', 'mqtt.retain', 'mqtt.sub.qos', 'mqtt.suback.qos', 'mqtt.ver', 'mqtt.willmsg', 'mqtt.willmsg_len', 'mqtt.willtopic', 'mqtt.willtopic_len', 'target']


In [28]:
# remoção das features que, visualmente, não são de forte influência nesse tipo de tráfego


# "to_dataset_dos" é o nome da variável apenas para a leitura dos dados sem as colunas irrelevantes, 
# pois posteriormente será necessária uma nova variável para ajustar as variáveis categóricas e tratar 
# os dados finais.

to_dataset_dos = data.drop(columns = ['mqtt.conack.flags.reserved', 'mqtt.conack.flags.sp', 'mqtt.conflag.qos', 'mqtt.conflag.reserved', 'mqtt.conflag.retain', 
             'mqtt.conflag.willflag', 'mqtt.retain', 'mqtt.sub.qos', 'mqtt.suback.qos', 'mqtt.willmsg', 'mqtt.willmsg_len', 
             'mqtt.willtopic', 'mqtt.willtopic_len', 'mqtt.msg'])

print(to_dataset_dos.shape)

(10000, 20)


In [31]:
# aplicação da função dummies para ajustar as variáveis categóricas com a definição
# da nova variável 


dataset_dos = pd.get_dummies(to_dataset_dos)


# novo tamanho do dataset com dummies

print(dataset_dos.shape)

(10000, 44)


Seleção de colunas com RFE

In [49]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR


X = dataset_dos.iloc[:, :-1]  # Seleciona todas as colunas exceto a última como características
y = dataset_dos.iloc[:, -1]   # Seleciona a última coluna como rótulo


estimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=10, step=1)
selector = selector.fit(X, y)
selector.support_

array([ True, False, False, False, False, False, False, False,  True,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True, False, False, False,
       False, False,  True,  True, False, False, False, False,  True,
       False, False, False, False, False, False,  True])

In [50]:
selector.ranking_

array([ 1,  8, 25, 16, 15, 14, 13, 11,  1,  1,  1, 19, 21, 24, 23, 22,  4,
       27, 29, 31, 30, 34,  1,  1,  3, 12, 10,  7, 18,  1,  1, 33, 32, 26,
        6,  1,  9, 17, 28, 20,  2,  5,  1])